# Assignment 3

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model]() or a linear model like [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) or [Ridge Regressors](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ridge_regression.html).
        - An advanced regressor of your choice (e.g., Bagging, Boosting, SVR, etc.). TIP: select a tree-based method such that it does not take too long to run SHAP further below. 

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Place the files in the ../../05_src/data/fires/ directory and load the appropriate file. 

In [5]:
import os
print(os.getcwd())


/Users/vishalbansur/production/02_activities/assignments


In [6]:
import pandas as pd

data = pd.read_csv('../../05_src/data/forestfires.csv')






In [8]:
# Load the data with default headers, then rename columns
fires_dt = pd.read_csv('../../05_src/data/forestfires.csv')

# Rename the columns after loading
fires_dt.columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]

# Verify the data
fires_dt.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   coord_x  517 non-null    int64  
 1   coord_y  517 non-null    int64  
 2   month    517 non-null    object 
 3   day      517 non-null    object 
 4   ffmc     517 non-null    float64
 5   dmc      517 non-null    float64
 6   dc       517 non-null    float64
 7   isi      517 non-null    float64
 8   temp     517 non-null    float64
 9   rh       517 non-null    int64  
 10  wind     517 non-null    float64
 11  rain     517 non-null    float64
 12  area     517 non-null    float64
dtypes: float64(8), int64(3), object(2)
memory usage: 52.6+ KB


# Get X and Y

Create the features data frame and target data.

In [ ]:
import pandas as pd

df = pd.read_csv('../../05_src/data/forestfires.csv')

# X (features) will include all columns except 'area' (which is the target)
X = df.drop('area', axis=1)



In [10]:
# Y (target) will be the 'area' column
Y = df['area']


# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PowerTransformer
from sklearn.impute import SimpleImputer
numeric_cols = X.select_dtypes(include=['float64', 'int64']).columns.tolist()
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()

# Preprocessing for preproc1: scaling and one-hot encoding
preproc1 = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_cols),   # Scaling the numerical columns using StandardScaler
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)  # One-hot encoding categorical columns
    ]
)

### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [14]:
# Preprocessing for preproc2: scaling, non-linear transformation, and one-hot encoding
preproc2 = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('scaler', StandardScaler()),             # Standard scaling
            ('power_transform', PowerTransformer())    # Power transformation (non-linear)
        ]), numeric_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)  # One-hot encoding categorical columns
    ]
)

# Fit and transform the data using preproc1 and preproc2
X_preproc1 = preproc1.fit_transform(X)
X_preproc2 = preproc2.fit_transform(X)


## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [40]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

# Example: Assuming you have numeric and categorical features
numeric_features = ['age', 'income', 'years_of_experience']  # List your numeric columns
categorical_features = ['gender', 'education_level']  # List your categorical columns

# Preprocessing for numeric data: imputation + scaling
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Fill missing values with mean
    ('scaler', StandardScaler())  # Standardize numeric features
])

# Preprocessing for categorical data: imputation + one-hot encoding
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Fill missing values with the most frequent category
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # One-hot encode categorical features
])

# Create the column transformer to apply the right preprocessing to the appropriate columns
preprocessor1 = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

# Baseline model: Linear Regression
baseline_regressor = LinearRegression()

# Advanced model: RandomForestRegressor (or you can choose another)
advanced_regressor = RandomForestRegressor()


In [ ]:
# Pipeline A = preproc1 + baseline
# Pipeline A: Preprocessing1 + Baseline
pipeline_a = Pipeline(steps=[
    ('preprocessor', preprocessor1),  # Preprocessing step
    ('regressor', baseline_regressor)  # Baseline regressor (Linear Regression)
])


In [42]:
# Pipeline B = preproc2 + baseline
# Pipeline B: Preprocessing2 + Baseline
# Here we assume a different preprocessing pipeline (e.g., different scaling/encoding)
# You can define preprocessor2 similarly as preprocessor1
# pipeline_b = Pipeline(steps=[
#    ('preprocessor', preprocessor2),  # Preprocessing step 2
#    ('regressor', baseline_regressor)  # Baseline regressor (Linear Regression)
# ])

In [43]:
# Pipeline C = preproc1 + advanced model
# Pipeline C: Preprocessing1 + Advanced Model (RandomForest)
pipeline_c = Pipeline(steps=[
    ('preprocessor', preprocessor1),  # Preprocessing step
    ('regressor', advanced_regressor)  # Advanced regressor (RandomForest)
])

In [44]:
# Pipeline D = preproc2 + advanced model

    # Pipeline D: Preprocessing2 + Advanced Model (RandomForest)
# pipeline_d = Pipeline(steps=[
#    ('preprocessor', preprocessor2),  # Preprocessing step 2
#    ('regressor', advanced_regressor)  # Advanced regressor (RandomForest)
# ])

# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [54]:
import pandas as pd

# Load the dataset
df = pd.read_csv('/Users/vishalbansur/production/02_activities/assignments/forestfires.csv')

# Define the feature matrix X and target variable y
X = df.drop('area', axis=1)  # Dropping 'area' to get features
y = df['area']  # Target variable

# Check that the setup was successful
print(X.head())  # Inspect the feature matrix
print(y.head())  # Inspect the target variable


   X  Y month  day  FFMC   DMC     DC  ISI  temp  RH  wind  rain
0  7  5   mar  fri  86.2  26.2   94.3  5.1   8.2  51   6.7   0.0
1  7  4   oct  tue  90.6  35.4  669.1  6.7  18.0  33   0.9   0.0
2  7  4   oct  sat  90.6  43.7  686.9  6.7  14.6  33   1.3   0.0
3  8  6   mar  fri  91.7  33.3   77.5  9.0   8.3  97   4.0   0.2
4  8  6   mar  sun  89.3  51.3  102.2  9.6  11.4  99   1.8   0.0
0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: area, dtype: float64


In [ ]:
# Load the data
df = pd.read_csv('/Users/vishalbansur/production/02_activities/assignments/forestfires.csv')

# Define X and y, with `area` as the target variable
X = df.drop('area', axis=1)
y = df['area']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [69]:
from sklearn.ensemble import RandomForestRegressor

# Use RandomForestRegressor instead of RandomForestClassifier
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor())
])


In [71]:
param_grid = {
    'regressor__n_estimators': [100, 200],
    'regressor__max_depth': [10, 20]
}


In [75]:
categorical_columns = ['month', 'day']
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Define the categorical columns
categorical_columns = ['month', 'day']  # Replace with your actual categorical columns

# Define the column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns)
    ]
)

# Now you can apply this preprocessor to your dataset


In [79]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Define the column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns)
    ]
)

In [77]:
# Define the numerical and categorical features
numerical_features = ['FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH', 'wind', 'rain']
categorical_features = ['month', 'day']

# Create the column transformer to preprocess the features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])


In [78]:
# Create the pipeline with RandomForestRegressor
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor())
])

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'regressor__n_estimators': [100, 200],   # Number of trees in the forest
    'regressor__max_depth': [10, 20]          # Max depth of each tree
}

In [84]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

# Sample columns for the example (you would replace these with actual column names from your dataset)
categorical_columns = ['month', 'day']  # Replace with actual categorical columns
numerical_columns = ['FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH', 'wind', 'rain']  # Replace with actual numerical columns

# Define preprocessing for numerical and categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_columns),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns)
    ]
)

# Define models for the pipelines (using regressors for a regression task)
models = {
    'linreg': LinearRegression(),
    'dtree': DecisionTreeRegressor(),
    'rf': RandomForestRegressor(),
    'svm': SVR()
}

# Define hyperparameters to tune for each model
param_grids = {
    'linreg': {'linreg__fit_intercept': [True, False]},
    'dtree': {'dtree__max_depth': [3, 5, 7, 10], 'dtree__min_samples_split': [2, 5, 10, 20]},
    'rf': {'rf__n_estimators': [50, 100, 200, 300], 'rf__max_depth': [3, 5, 10, 15]},
    'svm': {'svm__C': [0.1, 1, 10, 100], 'svm__kernel': ['linear', 'rbf']}
}

# Create a pipeline for each model and perform GridSearch
for model_name, model in models.items():
    # Build the pipeline for each model
    pipeline = Pipeline(steps=[('preprocessor', preprocessor), (model_name, model)])
    
    # Define the GridSearchCV for each pipeline with the corresponding parameter grid
    grid_search = GridSearchCV(pipeline, param_grids[model_name], cv=5, n_jobs=-1, verbose=2)
    
    print(f"Running GridSearch for {model_name}...")
    
    # Fit the GridSearchCV to your training data (replace X_train and y_train with your actual training data)
    grid_search.fit(X_train, y_train)  # Replace X_train and y_train with your actual training data
    
    print(f"Best parameters for {model_name}: {grid_search.best_params_}")
    print(f"Best score for {model_name}: {grid_search.best_score_}")


Running GridSearch for linreg...
Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV] END .........................linreg__fit_intercept=True; total time=   0.0s
[CV] END .........................linreg__fit_intercept=True; total time=   0.0s
[CV] END ........................linreg__fit_intercept=False; total time=   0.0s
[CV] END ........................linreg__fit_intercept=False; total time=   0.0s
[CV] END .........................linreg__fit_intercept=True; total time=   0.0s
[CV] END ........................linreg__fit_intercept=False; total time=   0.1s
[CV] END .........................linreg__fit_intercept=True; total time=   0.1s
[CV] END .........................linreg__fit_intercept=True; total time=   0.1s
[CV] END ........................linreg__fit_intercept=False; total time=   0.1s
[CV] END ........................linreg__fit_intercept=False; total time=   0.0s
Best parameters for linreg: {'linreg__fit_intercept': False}
Best score for linreg: -0.2921323457

# Evaluate

+ Which model has the best performance?

the linear regression model has the best performance

# Export

+ Save the best performing model to a pickle file.

In [89]:
# Split the data into features (X) and target (y)
X = data.drop(columns='area')  # Features (independent variables)
y = data['area']              # Target (dependent variable)

# Now, you can split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# You can now train a model using X_train, y_train


In [91]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# If you have columns with categorical data
categorical_columns = ['month', 'day']  # replace with the actual categorical column names

# Create a transformer for the categorical columns
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_columns)
    ], 
    remainder='passthrough'  # Keep other columns as they are
)

# Create a pipeline with preprocessing and model training
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', LinearRegression())
])

# Train the model using the pipeline
pipeline.fit(X_train, y_train)

# Make predictions
y_pred = pipeline.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse}')
print(f'R-squared (R2): {r2}')


Mean Squared Error (MSE): 11613.639591913126
R-squared (R2): 0.014771466120020338


# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

In [95]:
import shap
import matplotlib.pyplot as plt



In [99]:
from sklearn.preprocessing import LabelEncoder

# Initialize the encoder
le_month = LabelEncoder()
le_day = LabelEncoder()

# Apply LabelEncoder to both 'month' and 'day'
X_train['month'] = le_month.fit_transform(X_train['month'])
X_test['month'] = le_month.transform(X_test['month'])

X_train['day'] = le_day.fit_transform(X_train['day'])
X_test['day'] = le_day.transform(X_test['day'])


*(Answer here.)*

SHAP values provide a way to explain the output of machine learning models by attributing the prediction to the individual features. Each feature's SHAP value indicates how much that feature contributed to the difference between the predicted value and the mean predicted value. It helps us understand the model’s behavior and decision-making process for both individual predictions and across the entire dataset.
Using SHAP values, we can determine which features contributed most to the model's prediction for that observation. Positive SHAP values indicate features that pushed the prediction higher, while negative SHAP values indicate features that pushed the prediction lower.
SHAP values can be averaged across all observations in the dataset to give us a ranking of feature importance. This allows us to identify which features are most influential in making predictions across the entire training set.
To understand which features are actually enhancing the model’s performance, we can remove features that have low importance (based on SHAP values). We would then retrain the model and compare performance metrics (e.g., RMSE, R-squared) before and after removing these features.




## Criteria

The [rubric](./assignment_3_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-3`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_3.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.